In [1]:
%cd ..

/Users/erwan/Programmes/2022 Datacamp/solar_wind


#### Meilleurs modèles basés sur un traitement léger de Beta

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from problem import get_train_data, get_test_data
from problem import turn_prediction_to_event_list

# Fonction perso
from display import plot_event, multiple_plots, display_timeline, show_densities
from display import display_res


In [4]:
def get_fe_data(data):
    """
    Prend data_train['Beta'] au format brut (hors seuillabge ) et renvoie la série avec les traitements
    définis ci-dessous
    """

    # Moyenne mobile faible sur les valeurs pour espérer réduire dors et déjà le bruit 
    seuil = 50
    var_s = data.map(lambda x: min(x, seuil))
    var_s = var_s.rolling('30 min', center=True).mean()

    # Différence entre moyenne longue et moyenne courte (mobiles) 
    # Met en avant les transitions entre états
    var_l_2h_s = var_s.rolling('2h').mean()
    var_r_2h_s = var_s.iloc[::-1].rolling('2h').mean().iloc[::-1]

    # Ratio entre une moyenne mobile courte et une très longue
    # Met en avant les valeurs faibles de Beta
    var_4h_s = var_s.rolling('4h', center=True).mean()
    var_long_s = var_s.rolling('4 d', center=True).mean()

    var_l_20h_s = var_s.rolling('20h').mean()
    var_r_20h_s = var_s.iloc[::-1].rolling('20h').mean().iloc[::-1]

    df = pd.DataFrame({})
    df['base-value'] = var_s

    df['l_small_avg'] = var_l_2h_s.copy()
    df['r_small_avg'] = var_r_2h_s.copy()
    df['diff-mean'] = (df['r_small_avg'] - df['l_small_avg']).abs()

    df['mean-ratio'] = var_4h_s / var_long_s

    return df

In [5]:
# Données
data_train, labels_train = get_train_data()
data_test, labels_test = get_test_data()

# Évenements
events = turn_prediction_to_event_list(labels_train) 
no_events = turn_prediction_to_event_list(labels_train == 0)

In [6]:
fe_train_data = get_fe_data(data_train['Beta'])
fe_test_data = get_fe_data(data_test['Beta'])

In [7]:
# Ré-échantillone pour augmenter la proportion de 1
# (Pour cela on dégage directement des mesures correpondant à du calme
# Aucun traitement pour vérifier la cohérence des données après coup (en gros y'a des trous dans les données)
# -> Partie à réaliser pour le programme "Nettoyage des données" 
drop_fe_train = fe_train_data[labels_train == 0].sample(frac=0.55, random_state=1)

reduce_fe_data_train = fe_train_data.drop(drop_fe_train.index)
reduce_fe_labels_train = labels_train[reduce_fe_data_train.index]

In [9]:
reduce_fe_data_train.head(2)

,base-value,l_small_avg,r_small_avg,diff-mean,mean-ratio
1997-10-01 00:00:00,8.395598,8.395598,9.201718,0.806121,14.461996
1997-10-01 00:20:00,12.369014,10.112137,7.784553,2.327584,12.637000


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

In [14]:
cols = ['base-value', 'mean-ratio', 'diff-mean', 'l_small_avg', 'r_small_avg']

log_model = LogisticRegression(
    penalty='l2',
    max_iter=1000,
    random_state=0,
)

histboost_model = HistGradientBoostingClassifier(
    max_iter=1000,
    random_state=0,
)

print("Colonnes : ", cols)
print('\n Régression Logistique')
log_model.fit(reduce_fe_data_train[cols], reduce_fe_labels_train)
print('Coefs :', log_model.coef_)
display_res(fe_test_data[cols], labels_test, smooth=False, models=[log_model])


print("\n HistBoost")
histboost_model.fit(reduce_fe_data_train[cols], reduce_fe_labels_train)
display_res(fe_test_data[cols], labels_test, smooth=False, models=[histboost_model])

Colonnes :  ['base-value', 'mean-ratio', 'diff-mean', 'l_small_avg', 'r_small_avg']

 Régression Logistique
Coefs : [[ 1.24700003  0.26181994  5.53590306 -5.90231127 -5.8820308 ]]
Loss : 0.14837895513889757
              precision    recall  f1-score   support

           0       0.96      0.99      0.98    191755
           1       0.72      0.50      0.59     13819

    accuracy                           0.95    205574
   macro avg       0.84      0.74      0.78    205574
weighted avg       0.95      0.95      0.95    205574

ev_prec 0.5593220338983051
ev_rec 0.31775700934579443
-------------

 HistBoost
Loss : 0.13568117006308783
              precision    recall  f1-score   support

           0       0.96      0.99      0.98    191755
           1       0.72      0.50      0.59     13819

    accuracy                           0.95    205574
   macro avg       0.84      0.74      0.78    205574
weighted avg       0.95      0.95      0.95    205574

ev_prec 0.5964912280701755
ev_re